In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [2]:
# 1. KORELASI LD GURU DAN CT SISWA

# Load data
df_guru = pd.read_csv("files/data_guru_klasifikasi.csv")
df_siswa = pd.read_csv("files/data_siswa_klasifikasi.csv")

print("\nJumlah Guru per Level LD:")
print(df_guru["Level_LD"].value_counts())

print("\nJumlah Siswa per Level CT:")
print(df_siswa["Level_CT"].value_counts())



Jumlah Guru per Level LD:
Level_LD
Tinggi    59
Rendah    57
Sedang    56
Name: count, dtype: int64

Jumlah Siswa per Level CT:
Level_CT
Tinggi    563
Sedang    561
Rendah    554
Name: count, dtype: int64


In [3]:
# 2. CARI SEKOLAH YANG SAMA
def key_school(name):
    if pd.isna(name):
        return ""
    return " ".join(str(name).lower().strip().split()[:4])

df_guru["sekolah_key"] = df_guru["Asal Instansi"].apply(key_school)
df_siswa["sekolah_key"] = df_siswa["SekolahNama"].apply(key_school)

sekolah_sama = sorted(set(df_guru["sekolah_key"]) & set(df_siswa["sekolah_key"]))

print("Sekolah yang ada di kedua file:")
sekolah_sama

Sekolah yang ada di kedua file:


['sd kristen banjarsari',
 'sd kristen manahan surakarta',
 'sd kristen setabelan 1',
 'smk kristen 1 surakarta',
 'smp kristen 1 surakarta']

In [4]:
# 3. RATA-RATA PER SEKOLAH
df_guru_match = df_guru[df_guru["sekolah_key"].isin(sekolah_sama)]

ld_sekolah = (
    df_guru_match
    .groupby("sekolah_key")
    .agg(
        LD_Guru_Mean=("Mean_LD", "mean")
    )
    .round(3)
    .reset_index()
)

df_siswa_match = df_siswa[df_siswa["sekolah_key"].isin(sekolah_sama)]

ct_sekolah = (
    df_siswa_match
    .groupby("sekolah_key")
    .agg(
        CT_Siswa_Norm=("CT_norm", "mean")
    )
    .round(3)
    .reset_index()
)

df_korelasi = pd.merge(
    ld_sekolah,
    ct_sekolah,
    on="sekolah_key",
    how="inner"
)

print("\nData untuk Korelasi LD Guru dan CT Siswa per Sekolah:")
df_korelasi.round(3)




Data untuk Korelasi LD Guru dan CT Siswa per Sekolah:


,sekolah_key,LD_Guru_Mean,CT_Siswa_Norm
0,sd kristen banjarsari,3.535,0.355
1,sd kristen manahan surakarta,3.493,0.450
2,sd kristen setabelan 1,3.746,0.476
3,smk kristen 1 surakarta,3.363,0.228
4,smp kristen 1 surakarta,3.363,0.321


In [5]:
# Cek Korelasi
df_korelasi.describe()


,LD_Guru_Mean,CT_Siswa_Norm
count,5.000000,5.000000
mean,3.500000,0.366000
std,0.157582,0.100432
min,3.363000,0.228000
25%,3.363000,0.321000
50%,3.493000,0.355000
75%,3.535000,0.450000
max,3.746000,0.476000


In [6]:
# 3. HITUNG KORELASI SPEARMAN

In [7]:
# 3.1 Korelasi antar sekolah

# cari korelasi rata-rata di seluruh sekolah.
from scipy.stats import spearmanr

r, p = spearmanr(
    df_korelasi["LD_Guru_Mean"],
    df_korelasi["CT_Siswa_Norm"]
)

print("\n=== KORELASI SPEARMAN ANTAR SEKOLAH ===")
print(f"Korelasi Spearman (r): {r:.3f}")
print(f"p-value: {p:.3f}")



=== KORELASI SPEARMAN ANTAR SEKOLAH ===
Korelasi Spearman (r): 0.872
p-value: 0.054


In [8]:
# 3.2 Korelasi antar sekolah per level

from scipy.stats import spearmanr

for level in df_guru["Level_LD"].unique():
    print(f"\n===== LEVEL: {level} =====")

    # Filter per level
    guru_lvl = df_guru[df_guru["Level_LD"] == level]
    siswa_lvl = df_siswa[df_siswa["Level_CT"] == level]

    # Cari sekolah yang sama per level
    sekolah_guru = set(guru_lvl["sekolah_key"])
    sekolah_siswa = set(siswa_lvl["sekolah_key"])
    sekolah_sama_lvl = sorted(sekolah_guru & sekolah_siswa)

    print("Sekolah guru :", len(sekolah_guru))
    print("Sekolah siswa:", len(sekolah_siswa))
    print("Sekolah sama :", len(sekolah_sama_lvl))
    print("\nDaftar sekolah:")
    for s in sekolah_sama_lvl:
        print("-", s)

    # Hitung rata-rata per sekolah per level
    ld_per_sekolah = (
        guru_lvl[guru_lvl["sekolah_key"].isin(sekolah_sama_lvl)]
        .groupby("sekolah_key")["Mean_LD"]
        .mean()
        .reset_index(name="LD_Guru_Mean")
    )

    ct_per_sekolah = (
        siswa_lvl[siswa_lvl["sekolah_key"].isin(sekolah_sama_lvl)]
        .groupby("sekolah_key")["CT_norm"]
        .mean()
        .reset_index(name="CT_Siswa_Norm")
    )

    # Gabung data 
    df_lvl = pd.merge(ld_per_sekolah, ct_per_sekolah, on="sekolah_key")

    if len(df_lvl) >= 3:
        r, p = spearmanr(df_lvl["LD_Guru_Mean"], df_lvl["CT_Siswa_Norm"])
        print(f"\nKorelasi r = {r:.3f}, p = {p:.3f}")
    else:
        print("Data kurang, korelasi tidak dihitung")



===== LEVEL: Rendah =====
Sekolah guru : 10
Sekolah siswa: 11
Sekolah sama : 4

Daftar sekolah:
- sd kristen banjarsari
- sd kristen manahan surakarta
- smk kristen 1 surakarta
- smp kristen 1 surakarta

Korelasi r = 0.600, p = 0.400

===== LEVEL: Tinggi =====
Sekolah guru : 11
Sekolah siswa: 20
Sekolah sama : 4

Daftar sekolah:
- sd kristen banjarsari
- sd kristen manahan surakarta
- smk kristen 1 surakarta
- smp kristen 1 surakarta

Korelasi r = -0.400, p = 0.600

===== LEVEL: Sedang =====
Sekolah guru : 9
Sekolah siswa: 13
Sekolah sama : 4

Daftar sekolah:
- sd kristen banjarsari
- sd kristen manahan surakarta
- smk kristen 1 surakarta
- smp kristen 1 surakarta

Korelasi r = -0.800, p = 0.200


In [9]:
# 3.3 Korelasi per level 

# fokus lihat pola dalam sekolah.
from scipy.stats import pearsonr

hasil = []

for target in sekolah_sama:
    print(f"\n===== SEKOLAH: {target} =====")

    guru_filt = df_guru[df_guru["sekolah_key"] == target]
    siswa_filt = df_siswa[df_siswa["sekolah_key"] == target]

    print("Jumlah guru  :", len(guru_filt))
    print("Jumlah siswa :", len(siswa_filt))

    if len(guru_filt) == 0 or len(siswa_filt) == 0:
        print(">>> SKIP (data kurang)")
        hasil.append({
            "Sekolah": target,
            "r": None,
            "p": None,
            "Interpretasi": "Tidak bisa dihitung (data kurang)"
        })
        continue

    guru_per_level = (
        guru_filt.groupby("Level_LD")["Mean_LD"]
        .agg(["count", "mean"])
        .rename(columns={"count": "jumlah_guru", "mean": "mean_LD"})
        .reset_index()
    )
    siswa_per_level = (
        siswa_filt.groupby("Level_CT")["CT_norm"]
        .agg(["count", "mean"])
        .rename(columns={"count": "jumlah_siswa", "mean": "mean_CT"})
        .reset_index()
    )

    print("\nRata-rata LD Guru :")
    print(guru_per_level)

    print("\nRata-rata CT Siswa :")
    print(siswa_per_level)

    # Normalisasi huruf level supaya sama
    guru_per_level["Level"] = guru_per_level["Level_LD"].str.lower()
    siswa_per_level["Level"] = siswa_per_level["Level_CT"].str.lower()

    df_merge = pd.merge(
        guru_per_level[["Level", "mean_LD"]],
        siswa_per_level[["Level", "mean_CT"]],
        on="Level",
        how="inner"
    )

    print("\nDATA UNTUK KORELASI PER LEVEL")
    print(df_merge)

    if len(df_merge) >= 3:
        r, p = pearsonr(df_merge["mean_LD"], df_merge["mean_CT"])
        print(f"\nKorelasi r = {r:.3f}, p = {p:.3f}")

        interpret = "Kuat dan signifikan" if p < 0.05 else "Kuat tapi tidak signifikan"
        hasil.append({
            "Sekolah": target,
            "r": r,
            "p": p,
            "Interpretasi": interpret
        })
    else:
        print("\n=== HASIL KORELASI PER LEVEL ===")
        print("r : None")
        print("p : None")
        hasil.append({
            "Sekolah": target,
            "r": None,
            "p": None,
            "Interpretasi": "Tidak bisa dihitung (level kurang)"
        })



===== SEKOLAH: sd kristen banjarsari =====
Jumlah guru  : 19
Jumlah siswa : 89

Rata-rata LD Guru :
  Level_LD  jumlah_guru   mean_LD
0   Rendah            6  2.684833
1   Sedang            4  3.586750
2   Tinggi            9  4.078111

Rata-rata CT Siswa :
  Level_CT  jumlah_siswa   mean_CT
0   Rendah            29  0.185448
1   Sedang            31  0.338032
2   Tinggi            29  0.542103

DATA UNTUK KORELASI PER LEVEL
    Level   mean_LD   mean_CT
0  rendah  2.684833  0.185448
1  sedang  3.586750  0.338032
2  tinggi  4.078111  0.542103

Korelasi r = 0.968, p = 0.160

===== SEKOLAH: sd kristen manahan surakarta =====
Jumlah guru  : 35
Jumlah siswa : 219

Rata-rata LD Guru :
  Level_LD  jumlah_guru   mean_LD
0   Rendah           11  2.801818
1   Sedang           13  3.416154
2   Tinggi           11  4.273636

Rata-rata CT Siswa :
  Level_CT  jumlah_siswa   mean_CT
0   Rendah            40  0.183475
1   Sedang            65  0.347677
2   Tinggi           114  0.601167

DATA UNTUK 

In [10]:
# 4. TABEL RINGKAS
df_output = pd.DataFrame(hasil)
print("\n=== TABEL RINGKAS KORELASI PER SEKOLAH ===")
print(df_output.to_string(index=False))



=== TABEL RINGKAS KORELASI PER SEKOLAH ===
                     Sekolah        r        p                       Interpretasi
       sd kristen banjarsari 0.968498 0.160217         Kuat tapi tidak signifikan
sd kristen manahan surakarta 0.999616 0.017637                Kuat dan signifikan
      sd kristen setabelan 1      NaN      NaN Tidak bisa dihitung (level kurang)
     smk kristen 1 surakarta 0.999988 0.003159                Kuat dan signifikan
     smp kristen 1 surakarta 0.997380 0.046096                Kuat dan signifikan
